In [1]:
!pip install tensorflow-gpu
!pip install tensorflow-addons
!pip install ffmpeg

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu


C1 to C3

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import matplotlib
import ffmpeg
import gzip

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

class MMNISTDataset(tf.keras.utils.Sequence):

    def __init__(self, data_set):
        self.data_set = data_set
        self.batch_size = 16
        
        if data_set == "train":
          self.length = int(9008/self.batch_size)
          f = gzip.open('/Desktop/comp4211/pa2/pa2_data/train-images-idx3-ubyte.gz','r')
          image_size = 28
          num_images = 60000
          f.read(16)
          buf = f.read(image_size * image_size * num_images)
          self.mnist_train_im  = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
          self.mnist_train_im  = self.mnist_train_im.reshape(num_images, image_size, image_size, 1)
        elif data_set == "test":
          self.length = int(9008/self.batch_size)
          test_set = np.load('/Desktop/comp4211/pa2/pa2_data/MNIST_test_seq.npy')
          test_set = test_set.transpose(1,0,2,3)
          self.mnist_test = test_set[0:9008]
          self.mnist_test = np.expand_dims(self.mnist_test, axis=4)
        elif data_set == "valid":
          self.length = int(992/self.batch_size)
          test_set = np.load('/Desktop/comp4211/pa2/pa2_data/MNIST_test_seq.npy')
          test_set = test_set.transpose(1,0,2,3)
          self.mnist_valid = test_set[9008:10000]
          self.mnist_valid = np.expand_dims(self.mnist_valid, axis=4)

    def __len__(self):
        return self.length

    def gen_random_sequence(self):
        # randomly generate a sequence of a digit
        size = 64 - 28
        x, y, theta = np.random.random(), np.random.random(), np.random.random() * 2 * np.pi
        velocity_y, velocity_x = np.sin(theta), np.cos(theta)
        seq_x, seq_y = np.zeros(20), np.zeros(20)

        for i in range(20):
            y += 0.1*velocity_y
            x += 0.1*velocity_x
        
            if x <= 0: 
              x=0
              velocity_x = -velocity_x
            if x >= 1.0:
              x = 1.0
              velocity_x = -velocity_x
            if y <= 0:
              y=0
              velocity_y = -velocity_y
            if y >= 1.0:
              y = 1.0
              velocity_y = -velocity_y
            seq_x[i], seq_y[i] = x, y
            # Scale to the size.
            seq_x = (size * seq_x).astype(np.int32)
            seq_y = (size * seq_y).astype(np.int32)
            return seq_y, seq_x

    def random_mmnist(self):
        # generate frames of moving mnist
        data = np.zeros((20, 64, 64, 1), dtype=np.float32)
        # 10 input + 10 groundtruth = 20
        for n in range(2):
            seq_y, seq_x = self.gen_random_sequence()
            idx = np.random.randint(0, self.mnist_train_im.shape[0] - 1)
            mnist_image = self.mnist_train_im[idx]
            for i in range(20):
                # put the 2 moving digits into "data"
                data[i, seq_y[i]:seq_y[i]+28, seq_x[i]:seq_x[i]+28] =\
                    np.maximum(
                    data[i, seq_y[i]:seq_y[i]+28, seq_x[i]:seq_x[i]+28],
                    mnist_image
                    )
        return data

    def __getitem__(self, idx):
      if self.data_set == 'train':
        a = np.expand_dims(self.random_mmnist(), axis=0)
        for i in range(15):
          a = np.append(a, np.expand_dims(self.random_mmnist(), axis=0), axis=0)
        a = a/255
        input = a[:, 0:10]
        gt = a[:, 10:20]
      elif self.data_set == 'test':
        temp = self.mnist_test
        temp = temp/255
        input = temp[idx, 0:10]
        gt = temp[idx, 10:20]
      else:
        temp = self.mnist_valid
        temp = temp/255
        input = temp[idx, 0:10]
        gt = temp[idx, 10:20]

      return input, gt
      


2023-01-13 14:04:04.635560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'google'

C4

In [31]:
class Encoder(tf.keras.layers.Layer):

  def __init__(self):
    super(Encoder, self).__init__()
    self.skip_connection = None
    self.enc = [
        Enc_ConvB(64,(1,1)), # as the same H & W
        Enc_ConvB(64,(2,2)), # as output with H/2 & W/2
        Enc_ConvB(64,(1,1)), # as the same with above one
        Enc_ConvB(64,(2,2)) # as output with H/4 & W/4
    ]

  def call(self, x):
    x = tf.reshape(x, shape=(160, 64, 64, 1))
    x = self.enc[0](x)
    self.skip_connection = x
    x = self.enc[1](x)
    x = self.enc[2](x)
    x = self.enc[3](x)
    x = tf.reshape(x, shape=(16, 10, 64, 64, 1))
    return x

C5

In [32]:
class Enc_ConvB(tf.keras.layers.Layer):
  def __init__(self, out_channels_size, stride_size):
      super(Enc_ConvB, self).__init__()
      self.conv = tf.keras.layers.Conv2D(
          filters=out_channels_size, 
          strides=stride_size, 
          kernel_size=(3,3), 
          padding='same')
      self.norm = tfa.layers.GroupNormalization(groups=2, 
                                                epsilon=1e-5)
      self.act = tf.keras.layers.LeakyReLU(alpha=0.2)
    
  def call(self, x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.act(x)
    return x


Q1

s_1 = 1 as same H(64)&W(64) for output

s_2 = 2 as H/2(32)&W/2(32) for output

s_3 = 1 as same H/2(32)&W/2(32) for output

s_4 = 2 as H/4(16)&W/4(16) for output

e.g. [(64+2*1-3)/s_2 + 1] = 32, therefore s_2 = 2

Q2
The trainable parameters = 

C6

In [33]:
class Translator(tf.keras.layers.Layer):
  def __init__(self):
    super(Translator, self).__init__()
    self.enc = [
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256)
    ]
    self.dec = [
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(640)
    ]
  
  def call(self, x):
    x = x.transpose(0,3,1,2,4)
    x = tf.reshape(x, shape=(16, 16, 16, 10*64))
    IncepEnc_1 = self.enc[0](x)
    IncepEnc_2 = self.enc[1](IncepEnc_1,256)
    IncepEnc_3 = self.enc[2](IncepEnc_2,256)
    IncepEnc_4 = self.enc[3](IncepEnc_3,256)
    IncepEnc_5 = self.enc[4](IncepEnc_4,256)
    IncepEnc_6 = self.enc[5](IncepEnc_5,256)
    IncepEnc_7 = self.enc[6](IncepEnc_6,256)
    IncepEnc_8 = self.enc[7](IncepEnc_7,256)

    IncepDec_1 = self.dec[0](IncepEnc_8,256)
    temp = np.concatenate((IncepDec_1, IncepEnc_7), axis=3)
    IncepDec_2 = self.dec[1](temp,256)
    temp = np.concatenate((IncepDec_2, IncepEnc_6), axis=3)
    IncepDec_3 = self.dec[2](temp,256)
    temp = np.concatenate((IncepDec_3, IncepEnc_5), axis=3)
    IncepDec_4 = self.dec[3](temp,256)
    temp = np.concatenate((IncepDec_4, IncepEnc_4), axis=3)
    IncepDec_5 = self.dec[4](temp,256)
    temp = np.concatenate((IncepDec_5, IncepEnc_3), axis=3)
    IncepDec_6 = self.dec[5](temp,256)
    temp = np.concatenate((IncepDec_6, IncepEnc_2), axis=3)
    IncepDec_7 = self.dec[6](temp,256)
    temp = np.concatenate((IncepDec_7, IncepEnc_1), axis=3)
    IncepDec_8 = self.dec[7](temp,640)

    x = IncepDec_8.reshape(16,16,16,10,64)
    x = x.transpose(0,2,3,1,4)

    return x




C7

In [34]:
class Inception(tf.keras.layers.Layer):
  def __init__(self, C_out):
    super(Inception, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(
        filters=128, 
        strides=1, 
        kernel_size=(1,1)) # as the same H & W
    self.layers = [
        GroupConv(C_out,(3,3)),
        GroupConv(C_out,(5,5)),
        GroupConv(C_out,(7,7)),
        GroupConv(C_out,(11,11))
    ]

  def call(self, x):
    x = self.conv1(x)
    y1 = self.layers[0](x)
    y2 = self.layers[1](x)
    y3 = self.layers[2](x)
    y4 = self.layers[3](x)
    return (y1+y2+y3+y4)


Q3

The C_in:

for IncepEnc1 = 640

for IncepEnc2,3,4,5,6,7,8 & IncepDec1 = 256

for IncepDec2,3,4,5,6,7,8 = 512

Q4

The size should be 1 as output the same H'&W'

C8

In [35]:
class GroupConv(tf.keras.layers.Layer):
  def __init__(self, C_out, k):
    super(GroupConv, self).__init__()
    self.conv = tf.keras.layers.Conv2D(filters=C_out, strides=(1,1), kernel_size=k, padding='same', groups=8)
    self.norm = tfa.layers.GroupNormalization(groups=8, epsilon=1e-5)
    self.act = tf.keras.layers.LeakyReLU(alpha=0.2)
  
  def call(self, x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.act(x)
    return x

Q5

The number of times = 16*4 = 64

Q6

The C_out = 256 or 640
The kernal size = (3,3), (5,5), (7,7), (11,11)

C9

In [36]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, skip_connection):
    super(Decoder, self).__init__()
    self.skip_connection = skip_connection
    self.dec = [
        Dec_ConvB(64, (2,2), True), # double x & y
        Dec_ConvB(64, (1,1), False), # same x & y
        Dec_ConvB(64, (2,2), True), # double x & y
        Dec_ConvB(64, (1,1), False) # same x & y
    ]
    self.readout = tf.keras.layers.Conv2D(1, kernel_size = (1,1))
  
  def call(self, x):
    x = tf.reshape(x, shape=(16*10, 16, 16, 64))
    x = self.dec[0](x)
    x = self.dec[1](x)
    x = self.dec[2](x)
    x = np.concatenate((x, self.skip_connection), axis=3)
    x = self.dec[3](x)
    x = self.readout(x)
    x = tf.reshape(x, shape=(16, 10, 64, 64, 1))
    return x

C10

In [37]:
class Dec_ConvB(tf.keras.layers.Layer):
  def __init__(self, C_out, s, transpose):
    super(Dec_ConvB, self).__init__()
    self.transpose = transpose
    if transpose == True:
      self.conv = tf.keras.layers.Conv2DTranspose(filters=C_out, strides=s, kernel_size=(3,3), padding='same', output_padding = (1,1))
    else:
      self.conv = tf.keras.layers.Conv2D(filters=C_out, strides=s, kernel_size=(3,3), padding='same')
    self.norm = tfa.layers.GroupNormalization(groups=2, epsilon=1e-5)
    self.act = tf.keras.layers.LeakyReLU(alpha=0.2)

  def call(self, x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.act(x)
    return x


Q7

The values are 64

Q8

The s_1 = (2,2)
The s_2 = (1,1)
The s_3 = (2,2)
The s_4 = (1,1)

Q9

In [38]:
class Model(tf.keras.layers.Layer):
  def __init__(self):
    super(Model, self).__init__()
    self.enc = Encoder()
    self.hid = Translator()
    self.dec = Decoder(self.enc.skip_connection)
  
  def call(self, x):
    x = self.enc(x)
    x = self.hid(x)
    x = self.dec(x)


C11

In [39]:
model = tf.keras.Sequential(Model())

In [40]:
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=['mse', 'mae']
)

In [41]:
train_set = MMNISTDataset('train')
test_set = MMNISTDataset('test')
valid_set = MMNISTDataset('valid')

In [42]:
model.evaluate(test_set)

InvalidArgumentError: ignored

In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/pa2_data/pretrained_main/pretrained_main')

In [ ]:
model.evaluate(test_set)

In [ ]:
# model.fit(train_set, epochs=10, validation_data = valid_set)